# NYC Restaurant Health Inspection Data Cleaning

## 1. Import Libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 2. Load Dataset

In [13]:
# Load the dataset
file_path = '../data/DOHMH_New_York_City_Restaurant_Inspection_Results_20251105.csv'
df = pd.read_csv(file_path, low_memory=False)

print(df.info())
print(df.describe())
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291489 entries, 0 to 291488
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  291489 non-null  int64  
 1   DBA                    291486 non-null  object 
 2   BORO                   291489 non-null  object 
 3   BUILDING               290634 non-null  object 
 4   STREET                 291478 non-null  object 
 5   ZIPCODE                288351 non-null  float64
 6   PHONE                  291482 non-null  object 
 7   CUISINE DESCRIPTION    287573 non-null  object 
 8   INSPECTION DATE        291489 non-null  object 
 9   ACTION                 287573 non-null  object 
 10  VIOLATION CODE         285271 non-null  object 
 11  VIOLATION DESCRIPTION  285271 non-null  object 
 12  CRITICAL FLAG          291489 non-null  object 
 13  SCORE                  275118 non-null  float64
 14  GRADE                  142269 non-nu

# 3. Explore Dataset

In [14]:
df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location
0,50171397,COLD STONE CREAMERY,Queens,21265,26TH AVE,11360.0,5166953486,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,11/05/2025,NaN,40.778312,-73.777143,407.0,19.0,99704.0,4537937.0,4.059000e+09,QN47,POINT (-73.777142711788 40.778311801336)
1,50172126,ARK BAGELS,Manhattan,110,8 AVENUE,10011.0,3322050594,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,11/05/2025,NaN,40.740725,-74.001797,104.0,3.0,8100.0,1081538.0,1.007650e+09,MN13,POINT (-74.001797118593 40.740724556829)
2,50177069,CORNER BAR/SWAN ROOM,Manhattan,54-60,CANAL ST,10002.0,2106675921,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,11/05/2025,NaN,40.714904,-73.991970,103.0,1.0,1600.0,1003795.0,1.002940e+09,MN27,POINT (-73.991970220231 40.714904277236)
3,50143055,REN RESTAURANT,Brooklyn,1571,MCDONALD AVENUE,11230.0,9174561077,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,11/05/2025,NaN,40.614775,-73.974124,312.0,44.0,44600.0,3172689.0,3.065640e+09,BK88,POINT (-73.974124404003 40.614774895155)
4,50122424,ADMIRAL'S CLUB,Queens,NaN,TERMINAL B,11371.0,7184065129,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,11/05/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 4. Initial Filtering

Based on the dataset dictionary, we will:
1. **Drop unnecessary columns** not relevant to grade prediction
2. **Remove placeholder inspection dates** (01/01/1900)
3. **Keep only Cycle Inspections** - these are the regular health inspections that result in grades (A/B/C). Other inspection types (Smoke-Free Air Act, Inter-Agency Task Force, etc.) don't produce health grades.

In [ ]:
df_copy = df.copy()

# Drop unnecessary columns
drop_columns = ['PHONE', 'ACTION', 'RECORD DATE', 'Community Board', 'Council District', 
                'Census Tract', 'BIN', 'BBL', 'NTA', 'Location', 'Latitude', 'Longitude']
df_copy = df_copy.drop(columns=drop_columns)

print(f"Original shape: {df_copy.shape}")

# Remove placeholder inspection dates
drop_rows = df_copy[df_copy['INSPECTION DATE'] == '01/01/1900'].index
df_copy = df_copy.drop(index=drop_rows)
print(f"After removing placeholder dates: {df_copy.shape} (removed {len(drop_rows):,})")

# Keep only Cycle Inspections (the only ones that produce health grades)
before_count = len(df_copy)
df_copy = df_copy[df_copy['INSPECTION TYPE'].str.contains('Cycle Inspection', case=False, na=False)]
print(f"After filtering to Cycle Inspections only: {df_copy.shape} (removed {before_count - len(df_copy):,})")

df_copy.head()

Original shape: (291489, 15)
After removing placeholder dates: (287573, 15) (removed 3,916)
After filtering to Cycle Inspections only: (217809, 15) (removed 69,764)


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,INSPECTION TYPE
6,50075461,POPEYES,Manhattan,2300,ADAM CLAYTON POWELL JR BOULEVARD,NaN,Chicken,04/30/2024,09B,Thawing procedure improper.,Not Critical,22.0,NaN,NaN,Cycle Inspection / Initial Inspection
10,50000919,PANKO EXPRESS,Brooklyn,7802,FLATLANDS AVENUE,11236.0,Chinese,05/03/2024,09B,Thawing procedure improper.,Not Critical,12.0,A,05/03/2024,Cycle Inspection / Re-inspection
19,50063129,PUBLIC ARTS,Manhattan,215,CHRYSTIE ST/PUBLIC KITCHEN,NaN,American,06/24/2019,04A,Food Protection Certificate not held by superv...,Critical,10.0,A,06/24/2019,Cycle Inspection / Initial Inspection
20,50116166,GRAND STREET PIZZA,Manhattan,384,GRAND STREET,10002.0,Pizza,05/30/2023,NaN,NaN,Not Applicable,0.0,NaN,NaN,Cycle Inspection / Initial Inspection
33,40958170,SOHO HOUSE,Manhattan,2935,9 AVENUE,NaN,American,04/21/2023,08A,Establishment is not free of harborage or cond...,Not Critical,20.0,NaN,NaN,Cycle Inspection / Initial Inspection


# 5. Converting Data Types

In [16]:
# Convert date columns to datetime
df_copy['INSPECTION DATE'] = pd.to_datetime(df_copy['INSPECTION DATE'], format='%m/%d/%Y')
df_copy['GRADE DATE'] = pd.to_datetime(df_copy['GRADE DATE'], format='%m/%d/%Y', errors='coerce')

# Convert ZIPCODE from float to string (preserve leading zeros)
df_copy['ZIPCODE'] = df_copy['ZIPCODE'].astype('Int64').astype(str).replace('<NA>', None)

# Convert CAMIS to string (it's an ID, not a number)
df_copy['CAMIS'] = df_copy['CAMIS'].astype(str)

print("Data types after conversion:")
print(df_copy.dtypes)

Data types after conversion:
CAMIS                            object
DBA                              object
BORO                             object
BUILDING                         object
STREET                           object
ZIPCODE                          object
CUISINE DESCRIPTION              object
INSPECTION DATE          datetime64[ns]
VIOLATION CODE                   object
VIOLATION DESCRIPTION            object
CRITICAL FLAG                    object
SCORE                           float64
GRADE                            object
GRADE DATE               datetime64[ns]
INSPECTION TYPE                  object
dtype: object


# 6. Check Missing Values

In [17]:
# Check missing values
print("Missing values by column:")
print(df_copy.isnull().sum())
print(f"\nTotal rows: {len(df_copy):,}")

# Note: Some missing grades are expected for initial inspections that haven't been graded yet

Missing values by column:
CAMIS                         0
DBA                           0
BORO                          0
BUILDING                    674
STREET                        0
ZIPCODE                    2557
CUISINE DESCRIPTION           0
INSPECTION DATE               0
VIOLATION CODE              553
VIOLATION DESCRIPTION       553
CRITICAL FLAG                 0
SCORE                         0
GRADE                    102485
GRADE DATE               103824
INSPECTION TYPE               0
dtype: int64

Total rows: 217,809


# 7. Data Validation and Cleaning

In [18]:
# Trim whitespace from text columns
text_cols = ['DBA', 'STREET', 'BUILDING', 'CUISINE DESCRIPTION', 'VIOLATION DESCRIPTION']
for col in text_cols:
    if col in df_copy.columns:
        df_copy[col] = df_copy[col].str.strip()

print("\nData cleaning complete!")
print(f"Current shape: {df_copy.shape}")


Data cleaning complete!
Current shape: (217809, 15)


# 8. Check for Duplicates

In [19]:
# Check for duplicate rows
duplicates = df_copy.duplicated().sum()
print(f"Number of duplicate rows: {duplicates:,}")

if duplicates > 0:
    # Remove duplicates, keeping the first occurrence
    before_count = len(df_copy)
    df_copy = df_copy.drop_duplicates()
    print(f"Duplicates removed: {before_count - len(df_copy):,}")
    print(f"Final shape: {df_copy.shape}")
else:
    print("No duplicates found.")

Number of duplicate rows: 6
Duplicates removed: 6
Final shape: (217803, 15)


In [20]:
# Analyze key distributions
print("INSPECTION TYPE DISTRIBUTION:")
print(df_copy['INSPECTION TYPE'].value_counts())

print("\n" + "="*60)
print("GRADE DISTRIBUTION:")
grade_counts = df_copy['GRADE'].value_counts().sort_index()
print(grade_counts)
print(f"\nGrade missing: {df_copy['GRADE'].isna().sum():,} ({df_copy['GRADE'].isna().sum()/len(df_copy)*100:.1f}%)")

print("\n" + "="*60)
print("DATE RANGE:")
print(f"Earliest inspection: {df_copy['INSPECTION DATE'].min()}")
print(f"Latest inspection: {df_copy['INSPECTION DATE'].max()}")

print("\n" + "="*60)
print("TOP 10 CUISINES:")
print(df_copy['CUISINE DESCRIPTION'].value_counts().head(10))

print("\n" + "="*60)
print("BOROUGH DISTRIBUTION:")
print(df_copy['BORO'].value_counts())

INSPECTION TYPE DISTRIBUTION:
INSPECTION TYPE
Cycle Inspection / Initial Inspection              153229
Cycle Inspection / Re-inspection                    60351
Cycle Inspection / Reopening Inspection              2792
Cycle Inspection / Compliance Inspection             1344
Cycle Inspection / Second Compliance Inspection        87
Name: count, dtype: int64

GRADE DISTRIBUTION:
GRADE
A    82073
B    15270
C    10823
N     1339
P      745
Z     5074
Name: count, dtype: int64

Grade missing: 102,479 (47.1%)

DATE RANGE:
Earliest inspection: 2015-09-24 00:00:00
Latest inspection: 2025-11-03 00:00:00

TOP 10 CUISINES:
CUISINE DESCRIPTION
American                    36332
Chinese                     21169
Coffee/Tea                  14161
Pizza                       13494
Latin American              10422
Mexican                      8998
Bakery Products/Desserts     8968
Caribbean                    8488
Italian                      7856
Japanese                     7189
Name: count, dty

In [ ]:
# Check zero scores - are they legitimate or suspicious?
zero_scores = df_copy[df_copy['SCORE'] == 0]
print(f"Records with SCORE = 0: {len(zero_scores):,}")
print(f"\nGrades for zero scores:")
print(zero_scores['GRADE'].value_counts(dropna=False))
print(f"\nInspection types for zero scores:")
print(zero_scores['INSPECTION TYPE'].value_counts())
print(f"\nSample of zero score records:")
print(zero_scores[['DBA', 'SCORE', 'GRADE', 'INSPECTION DATE', 'INSPECTION TYPE']].head(10))

In [21]:
# Analyze key distributions
print("INSPECTION TYPE DISTRIBUTION:")
print(df_copy['INSPECTION TYPE'].value_counts())

print("\n" + "="*60)
print("GRADE DISTRIBUTION:")
grade_counts = df_copy['GRADE'].value_counts().sort_index()
print(grade_counts)
print(f"\nGrade missing: {df_copy['GRADE'].isna().sum():,} ({df_copy['GRADE'].isna().sum()/len(df_copy)*100:.1f}%)")

print("\n" + "="*60)
print("DATE RANGE:")
print(f"Earliest inspection: {df_copy['INSPECTION DATE'].min()}")
print(f"Latest inspection: {df_copy['INSPECTION DATE'].max()}")

print("\n" + "="*60)
print("TOP 10 CUISINES:")
print(df_copy['CUISINE DESCRIPTION'].value_counts().head(10))

print("\n" + "="*60)
print("BOROUGH DISTRIBUTION:")
print(df_copy['BORO'].value_counts())

INSPECTION TYPE DISTRIBUTION:
INSPECTION TYPE
Cycle Inspection / Initial Inspection              153229
Cycle Inspection / Re-inspection                    60351
Cycle Inspection / Reopening Inspection              2792
Cycle Inspection / Compliance Inspection             1344
Cycle Inspection / Second Compliance Inspection        87
Name: count, dtype: int64

GRADE DISTRIBUTION:
GRADE
A    82073
B    15270
C    10823
N     1339
P      745
Z     5074
Name: count, dtype: int64

Grade missing: 102,479 (47.1%)

DATE RANGE:
Earliest inspection: 2015-09-24 00:00:00
Latest inspection: 2025-11-03 00:00:00

TOP 10 CUISINES:
CUISINE DESCRIPTION
American                    36332
Chinese                     21169
Coffee/Tea                  14161
Pizza                       13494
Latin American              10422
Mexican                      8998
Bakery Products/Desserts     8968
Caribbean                    8488
Italian                      7856
Japanese                     7189
Name: count, dty

# 10. Export Cleaned Data

In [ ]:
# Final summary
print("=" * 60)
print("FINAL CLEANED DATASET")
print("=" * 60)
print(f"Shape: {df_copy.shape}")
print(f"Columns: {list(df_copy.columns)}")
print(f"\nMissing values:")
missing = df_copy.isnull().sum()
print(missing[missing > 0])

print("\nSample:")
print(df_copy.head(3))

# Export to CSV
output_path = '../data/cleaned_restaurant_inspections.csv'
df_copy.to_csv(output_path, index=False)
print(f"\n✓ Exported to: {output_path}")

df_copy.head()

FINAL CLEANED DATASET
Shape: (217803, 15)
Columns: ['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'CUISINE DESCRIPTION', 'INSPECTION DATE', 'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE', 'GRADE DATE', 'INSPECTION TYPE']

Missing values:
BUILDING                    674
ZIPCODE                    2551
VIOLATION CODE              553
VIOLATION DESCRIPTION       553
GRADE                    102479
GRADE DATE               103818
dtype: int64

Sample:
       CAMIS            DBA       BORO BUILDING  \
6   50075461        POPEYES  Manhattan     2300   
10  50000919  PANKO EXPRESS   Brooklyn     7802   
19  50063129    PUBLIC ARTS  Manhattan      215   

                              STREET ZIPCODE CUISINE DESCRIPTION  \
6   ADAM CLAYTON POWELL JR BOULEVARD    None             Chicken   
10                  FLATLANDS AVENUE   11236             Chinese   
19        CHRYSTIE ST/PUBLIC KITCHEN    None            American   

   INSPECTION DATE VIOLATION 

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,INSPECTION TYPE
6,50075461,POPEYES,Manhattan,2300,ADAM CLAYTON POWELL JR BOULEVARD,None,Chicken,2024-04-30,09B,Thawing procedure improper.,Not Critical,22.0,NaN,NaT,Cycle Inspection / Initial Inspection
10,50000919,PANKO EXPRESS,Brooklyn,7802,FLATLANDS AVENUE,11236,Chinese,2024-05-03,09B,Thawing procedure improper.,Not Critical,12.0,A,2024-05-03,Cycle Inspection / Re-inspection
19,50063129,PUBLIC ARTS,Manhattan,215,CHRYSTIE ST/PUBLIC KITCHEN,None,American,2019-06-24,04A,Food Protection Certificate not held by superv...,Critical,10.0,A,2019-06-24,Cycle Inspection / Initial Inspection
20,50116166,GRAND STREET PIZZA,Manhattan,384,GRAND STREET,10002,Pizza,2023-05-30,NaN,NaN,Not Applicable,0.0,NaN,NaT,Cycle Inspection / Initial Inspection
33,40958170,SOHO HOUSE,Manhattan,2935,9 AVENUE,None,American,2023-04-21,08A,Establishment is not free of harborage or cond...,Not Critical,20.0,NaN,NaT,Cycle Inspection / Initial Inspection
